# ML Technique - SVM

Sources: https://github.com/cmadusankahw/email-spam-detection-with-SVM
https://www.kaggle.com/code/elnahas/phishing-email-detection-using-svm-rfc

In [12]:
#Importing the necessary libraries
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

